In [ ]:
# 연습..!

In [ ]:
import os
import sys
import urllib.request
import json
import pandas as pd
import re

# 셀레니움 임포트
from selenium import webdriver
import time

# chrome 브라우저 이용할 것
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# 원하는 내용이 포함된 DF에 댓글 링크 포함하는 함수
# '네이버 뉴스만 포함'시켜서 DF를 만들면 사용 가능

def add_comment_link(df):
    for i in range(df.shape[0]):
        # 변수에 링크 저장
        news_link = df.loc[i, 'Link']
        # / 로 news_link 나누기
        news_list = news_link.split('/')
        # 'comment' 삽입하기
        news_list.insert(5, 'comment')
        # 'news_list'를 다시 합치기
        comment_link = '/'.join(news_list)
        # DF에 댓글 링크 컬럼 추가하고 링크 넣기
        df.loc[i, 'CommentLink'] = comment_link
    # DF return
    return df

In [ ]:
# 댓글 내용 가져오기 함수
# 댓글이 19개 밖에 가져와지지 않음.. 이거 해결해야 함.

def get_comments(df):
    new_df = pd.DataFrame({'Title' : df['name'].to_list()})
    # df의 행 수 만큼 반복
    for i in range(df.shape[0]):
        # 댓글을 찾고 싶은 페이지 열
        comment_link = df.loc[i, 'CommentLink']
        driver.get(comment_link)
        # comment를 담을 리스트 만들기
        comments_list = []
        # 크롬에서 댓글 긁어오기
        comments = driver.find_elements(By.CSS_SELECTOR, 'span.u_cbox_contents')
        # for문 이용해서 'comments_list'에 댓글 넣기
        for j, comment in enumerate(comments):
            comments_list.append(comment.text)
            new_df.loc[i, f'comment{j+1}'] = comment.text
    print(len(comments_list))

    return new_df

## 네이버 뉴스에서 '라인 사태'에 관한 뉴스 가져오기

In [ ]:
# client_id, client_secret : 본인 인증키 넣기
client_id = "CGI0TWpzp1hxoq5hfXj8"
client_secret = "pd8aH97a5e"
query = urllib.parse.quote(input('검색 질의 : '))   #검색어 입력받음

idx = 0
display = 100    # 한 번에 보여줄 검색결과 개수
start = 1   # 검색 시작 인덱스
end = 1000   # 검색 끝 인덱스
sort = 'sim'    # 정확도순 내림차순 정렬 (date: 날짜순 내림차순 정렬)

news_df = pd.DataFrame(columns=['Title', 'Original Link', 'Link', 'Description', 'Publication Date'])

# 1 ~ 1000까지 100 간격으로 돌아감 (1 ~ 100, 101 ~ 200, ... , 901 ~ 1000)
for i in range(start, end, display):
    url = 'https://openapi.naver.com/v1/search/news.json?'
    url += ('query=' + query)
    url += ('&display=' + str(display))
    url += ('&start=' + str(i))
    url += ('&sort=' + sort)

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if (rescode == 200):
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']

        for item_index in range(0, len(items)):   # 각 기사의 정보가 리스트로 들어가 있음
            remove_tag = re.compile('<.*?>')   # 태그 제거

            title = re.sub(remove_tag, '', items[item_index]['title'])   # 리스트 안에는 딕셔너리 형태
            original_link = items[item_index]['originallink']
            link = items[item_index]['link']
            description = re.sub(remove_tag, '', items[item_index]['description'])
            pub_date = items[item_index]['pubDate']
            news_df.loc[idx] = [title, original_link, link, description, pub_date]   # 데이터프레임에 삽입
            idx += 1

    else:
        print("Error Code: " + rescode)

# # 네이버 뉴스 외의 뉴스 링크도 포함
# news_df

# 네이버 뉴스만 포함
news_df = news_df[news_df['Link'].str.contains('https://n.news.naver.com/mnews/article/')].reset_index(drop=True)

## news_df의 'Link'에서 댓글 찾아서 댓글 링크 가져오기 + news_df에 컬럼 추가 (아직 시도중..)
* 'Link' 돌면서 댓글 가져오면 될 것 같은데..

In [ ]:
import pandas as pd

df_ex = pd.DataFrame({'name' : [1],
              'CommentLink' : ['https://n.news.naver.com/mnews/article/comment/079/0003894645?sid=100']})

In [ ]:
# 브라우저 옵션 설정
chrome_options = Options()

# 창 닫기
chrome_options.add_argument('--headless')

# detach 옵션을 이용해 자동화
# 브라우저를 자동화한 후, browser window 창 유지
# chrome_options.add_experimental_option("detach", True)

# excludeSwitches : 불필요한 로깅 메세지 브라우저에서 제외
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 셀레니움에서 크롬 웹 드라이버를 자동으로 다운로드 및 설치
service = Service(executable_path=ChromeDriverManager().install())

# 웹드라이버 설정
driver = webdriver.Chrome(service=service, options=chrome_options)

# 대기시간 : 10초
# wait = WebDriverWait(driver, 10)
# s = Service(path)
# driver = webdriver.Chrome()
driver.implicitly_wait(10)
# driver.set_window_size(1600, 800)

get_comments(df_ex)


19


,Title,comment1,comment2,comment3,comment4,comment5,comment6,comment7,comment8,comment9,comment10,comment11,comment12,comment13,comment14,comment15,comment16,comment17,comment18,comment19
0,1,이상한 제목 달지 말고 그냥 일본정부가 말도 안 되는 짓 당장 그만두게 하고 네이버...,여기 김형준이는 이미 기울어진 기사 제목으로 장난치고 있네... 이런 외교 갈등 문...,한국인이 한국땅 가는게 반일이냐,정부가 인식이. 없어니. 야당이. 혼을 내는건 당연. 동쪽구김당은 왜구당 일듯,걍 강원도와 경상도는 일본에 줘 버렸으면 좋겠다....시끌럽게 이러지 말고.....,본질이 강탈이지 뭔 소리냐,일본이 행정지도한지 두달이 지나서야? 겨우 여론이 악화되니 유감표명만 하는 무능한 ...,비판조차 안하는 정부와 여당이 문제아닌가? 대한민국 정체성마저 없는데 사기업간의 일...,대안을 정부에서 찾아야지..야당 주장엔 무조건 거부하는 윤석열에 밖에서 제시하는 어...,일본이 저런짓을 하는데 당연히 반일감정이 생기지 안생기겠냐? 친일 매국노 국짐당아.,기 렉 ㆍ 친일파들 눈 과 입은 에는 반일 타령만 ㅠㅠ 독도 와 라인비롯 경제까지 ...,대안을 야당에서 내놓으면 무능한 정부는 모두 사퇴하고 정권이양해라,왜 한국정부는 일본정부에게 항의조차 못하고 있나?,대안? 기자야 대안을 왜 야당이 해야되 니네 정부랑 여당이 해야기사로 물타기좀 그만해,ㅇㅋ 그럼 정부의 대책은 무엇인가요? 누군가 나서서 돈까스라도 먹어야 하는거 아닌가요?,니들 처럼 친일 해서야 대안이 나오겠냐 ? 요점은 일의 침략성에 경각심을 갖고 경계...,한국에도 쿠팡이랑 롯데가 있고 또 여러 일본 기업있더만... 국제적으로 걸리는거 붉...,'히로부미 손자요' 하면 일본 정부가 '어떻게 알았지? 안되겠다 포기하자' 이렇게 ...,아무 대책없는 정부에 야당이 일제히 팔면 안된다고 주장하고 여론을 형성하는게 대안 ...


In [ ]:
### 위의 결과는 'df_ex' 사용한 것입니다.